<a href="https://colab.research.google.com/github/peremartra/FinLLMOpt/blob/FinChat-XS-Instruct/Llama-FinChat-S/01_Finetuning_FinChat-XS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FinChat-XS
## Fine-Tuning & Merge.


________
* Model: HuggingFaceTB/SmolLM2-360M-Instruct
* Dataset: sujet-ai/Sujet-Finance-Instruct-177k
_________
This notebook simply replicates the process followed to create the Llama-FinSent-S model.

If you’re looking for explanations of each step, you can find them in the Pruning section of the [Large Language Models course](https://github.com/peremartra/Large-Language-Model-Notebooks-Course) that I maintain on GitHub.

# Install and Import Libraries.

In [1]:
!pip install -q transformers==4.47.1
!pip install -q datasets==3.2.0
!pip install -q torch==2.5.1
!pip install -q lm-eval==0.4.7
!pip install -q peft==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [2]:
# Import Libraries (updated)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
from transformers import DataCollatorForLanguageModeling
import re
from transformers import EarlyStoppingCallback

# Load & Treat the Dataset

In [3]:
# Load the Dataset
dataset = load_dataset("sujet-ai/Sujet-Finance-Instruct-177k")
dataset["train"] = dataset["train"].filter(lambda x: x["task_type"].strip().lower() == "qa")

README.md:   0%|          | 0.00/4.64k [00:00<?, ?B/s]

Sujet-Finance-Instruct-177k.csv:   0%|          | 0.00/337M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177597 [00:00<?, ? examples/s]

Filter:   0%|          | 0/177597 [00:00<?, ? examples/s]

In [4]:
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=2)
# split_dataset now has keys: "train" and "test"

In [5]:
split_dataset["train"][10000]

{'Unnamed: 0': 123865,
 'inputs': 'As a finance expert, your role is to provide clear, concise, and informative responses to finance-related questions. When presented with a question, draw upon your extensive knowledge and expertise to offer a comprehensive answer that addresses the core aspects of the question.\n\nQuestion:\nGenerate a list of 5 arguments for government regulation of business.\n\nAnswer:',
 'answer': '1. To protect consumers from harm from unsafe or unethical business practices\n2. To create a fair playing field by preventing companies from gaining too much market power\n3. To help ensure that businesses are acting in the public interest\n4. To provide safeguards to promote competition and innovation\n5. To minimize the potential for conflicts of interest.',
 'system_prompt': 'As a finance expert, your role is to provide clear, concise, and informative responses to finance-related questions. When presented with a question, draw upon your extensive knowledge and expert

In [6]:
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'inputs', 'answer', 'system_prompt', 'user_prompt', 'task_type', 'dataset', 'index_level', 'conversation_id'],
        num_rows: 34920
    })
    test: Dataset({
        features: ['Unnamed: 0', 'inputs', 'answer', 'system_prompt', 'user_prompt', 'task_type', 'dataset', 'index_level', 'conversation_id'],
        num_rows: 3881
    })
})


## Load Tokenizer & Model

In [7]:
# Load the Base Model and Tokenizer.
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 960, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=960, out_features=960, bias=False)
          (k_proj): Linear(in_features=960, out_features=320, bias=False)
          (v_proj): Linear(in_features=960, out_features=320, bias=False)
          (o_proj): Linear(in_features=960, out_features=960, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=960, out_features=2560, bias=False)
          (up_proj): Linear(in_features=960, out_features=2560, bias=False)
          (down_proj): Linear(in_features=2560, out_features=960, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((960,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((960,), eps=1e-05)
      )
    )
    (norm

In [9]:
print(model.dtype)

torch.bfloat16


## Format Prompt for Training.

In [11]:
def format_chat(row):
    user_prompt = row["user_prompt"]

    if re.match(r'^\s*question\b', user_prompt, re.IGNORECASE):
        # Remove everything up to the first colon
        user_prompt = re.sub(r'^\s*question.*?:\s*', '', user_prompt, flags=re.IGNORECASE)

    messages = [
        {"role": "system", "content": """You are FinChat, a specialized finance AI assistant. Provide accurate,
concise information on markets, investments, accounting, and personal finance.
Always clarify when financial information may vary by jurisdiction."""},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": row["answer"]}
    ]
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

In [12]:
# Apply formatting
formatted_dataset = split_dataset.map(format_chat)

Map:   0%|          | 0/34920 [00:00<?, ? examples/s]

Map:   0%|          | 0/3881 [00:00<?, ? examples/s]

In [13]:
print(formatted_dataset["test"][100]["text"])

<|im_start|>system
You are FinChat, a specialized finance AI assistant. Provide accurate, 
concise information on markets, investments, accounting, and personal finance. 
Always clarify when financial information may vary by jurisdiction.<|im_end|>
<|im_start|>user
Design an algorithm to detect plagiarism in academic papers.<|im_end|>
<|im_start|>assistant
The algorithm should cross-reference a given paper with a library of academic papers and sources to compare for any shared content. If detected, the algorithm should be able to calculate a plagiarism score which could be used to rate the authors’ work. Additionally, the algorithm should recognize when sources are properly cited and offer feedback on the accuracy of the citations.<|im_end|>



## Tokenize Dataset.

In [14]:
# Tokenize the dataset (truncating to a maximum length, e.g., 512 tokens).
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=768)

In [15]:
cols_to_remove = formatted_dataset["train"].column_names
tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True, remove_columns=cols_to_remove)


Map:   0%|          | 0/34920 [00:00<?, ? examples/s]

Map:   0%|          | 0/3881 [00:00<?, ? examples/s]

# Training

In [16]:
# LoRA Configuration (unchanged)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.3,
    #target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
model_peft = get_peft_model(model, lora_config)

In [17]:
print(model_peft.print_trainable_parameters())

trainable params: 1,638,400 || all params: 363,459,520 || trainable%: 0.4508
None


In [18]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./lora_finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    #fp16=True,
    bf16=True,
    weight_decay=0.001,
    warmup_steps=200,
    save_steps=200,
    logging_steps=200,
    evaluation_strategy="steps",
    report_to="none",
    # Add this to handle padding properly:
    dataloader_pin_memory=False,
    max_grad_norm=0.5,
    lr_scheduler_type="cosine",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    remove_unused_columns=False  # Required for chat templates
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# Use proper data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8  # Improve GPU efficiency
)

trainer = Trainer(
    model=model_peft,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [20]:
# Execute Training
trainer.train()
print("Training complete!")


Step,Training Loss,Validation Loss
200,2.456200,2.356586
400,2.113300,1.866798
600,1.602100,1.470255
800,1.468500,1.451024
1000,1.454000,1.440855
1200,1.456700,1.434151
1400,1.427400,1.430036
1600,1.432500,1.427428
1800,1.440200,1.425933
2000,1.445900,1.425436


Training complete!


## Save & Upload Model to HF

In [21]:
new_model_name = "FinChat6-XS"
# Merge the LoRA Adapter with the base model.
merged_model = model_peft.merge_and_unload()
merged_model.save_pretrained(new_model_name)

In [22]:
merged_model.push_to_hub(new_model_name,
                  private=True,
                  use_temp_dir=False)
tokenizer.push_to_hub(new_model_name,
                      private=True,
                      use_temp_dir=False)

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/oopere/FinChat6-XS/commit/c7121bd88bc5ab5d740ada2444985c88b3959fc3', commit_message='Upload tokenizer', commit_description='', oid='c7121bd88bc5ab5d740ada2444985c88b3959fc3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/oopere/FinChat6-XS', endpoint='https://huggingface.co', repo_type='model', repo_id='oopere/FinChat6-XS'), pr_revision=None, pr_num=None)

In [24]:
system_prompt = """You are FinChat, a specialized finance AI assistant. Provide accurate,
concise information on markets, investments, accounting, and personal finance.
Always clarify when financial information may vary by jurisdiction. """
# After training - evaluate on finance-specific test cases
test_prompts = [
    "Explain the difference between stocks and bonds",
    "What factors affect interest rates?",
    "Describe the key components of a balance sheet"
]

for prompt in test_prompts:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    model_input = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = merged_model.generate(
            input_ids=model_input,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True
        )
    response = tokenizer.decode(outputs[0][model_input.shape[1]:], skip_special_tokens=True)
    print(f"Prompt: {prompt}\nResponse: {response}\n")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Prompt: Explain the difference between stocks and bonds
Response: assistant
Stocks and bonds are two types of investments that are traded on the stock market. The main difference between them is how you earn money from them and how the money is invested. 

Stocks are short for "shares" and are the most common form of investment. You purchase shares of stock and then you become a part-owner of the company that issued them. When you sell your stock, you sell the shares you owned. 

Bonds are short for "debt" and are issued by a government or corporation. When you buy a bond, you are loaned money by the issuer to its investors. The issuer pays back the loan, usually with interest, over a set period of time. 

The money you invest is often used to fund public projects, such as roads, schools, and hospitals. The interest you earn on your bond may be paid out in cash or through a bank account. 
When you own a bond

Prompt: What factors affect interest rates?
Response: assistant
Interest rate

In [22]:
# Push adapter to Hub
model_peft.push_to_hub(
    "qa-adapter" + new_model_name,
    commit_message="Add qa LoRA adapter"
)

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/oopere/qa-adapterFinChat4-XS/commit/8e50116003f72d53e557aa91a4220cbe1089bc44', commit_message='Add qa LoRA adapter', commit_description='', oid='8e50116003f72d53e557aa91a4220cbe1089bc44', pr_url=None, repo_url=RepoUrl('https://huggingface.co/oopere/qa-adapterFinChat4-XS', endpoint='https://huggingface.co', repo_type='model', repo_id='oopere/qa-adapterFinChat4-XS'), pr_revision=None, pr_num=None)

# Conclusions

Our financial sentiment analysis model is now ready.

The entire process took just over 40 minutes on a Google Colab A100 GPU, which is roughly equivalent to an RTX 3090.
